# **Day20**: *Pulse Propagation*

## Part 1

With your help, the Elves manage to find the right parts and fix all of the machines. Now, they just need to send the command to boot up the machines and get the sand flowing again.

The machines are far apart and wired together with long cables. The cables don't connect to the machines directly, but rather to communication modules attached to the machines that perform various initialization tasks and also act as communication relays.

Modules communicate using pulses. Each pulse is either a high pulse or a low pulse. When a module sends a pulse, it sends that type of pulse to each module in its list of destination modules.

There are several different types of modules:

Flip-flop modules (prefix %) are either on or off; they are initially off. If a flip-flop module receives a high pulse, it is ignored and nothing happens. However, if a flip-flop module receives a low pulse, it flips between on and off. If it was off, it turns on and sends a high pulse. If it was on, it turns off and sends a low pulse.

Conjunction modules (prefix &) remember the type of the most recent pulse received from each of their connected input modules; they initially default to remembering a low pulse for each input. When a pulse is received, the conjunction module first updates its memory for that input. Then, if it remembers high pulses for all inputs, it sends a low pulse; otherwise, it sends a high pulse.

There is a single broadcast module (named broadcaster). When it receives a pulse, it sends the same pulse to all of its destination modules.

Here at Desert Machine Headquarters, there is a module with a single button on it called, aptly, the button module. When you push the button, a single low pulse is sent directly to the broadcaster module.

After pushing the button, you must wait until all pulses have been delivered and fully handled before pushing it again. Never push the button if modules are still processing pulses.

Pulses are always processed in the order they are sent. So, if a pulse is sent to modules a, b, and c, and then module a processes its pulse and sends more pulses, the pulses sent to modules b and c would have to be handled first.

The module configuration (your puzzle input) lists each module. The name of the module is preceded by a symbol identifying its type, if any. The name is then followed by an arrow and a list of its destination modules. For example:

```
broadcaster -> a, b, c
%a -> b
%b -> c
%c -> inv
&inv -> a
```

In this module configuration, the broadcaster has three destination modules named a, b, and c. Each of these modules is a flip-flop module (as indicated by the % prefix). a outputs to b which outputs to c which outputs to another module named inv. inv is a conjunction module (as indicated by the & prefix) which, because it has only one input, acts like an inverter (it sends the opposite of the pulse type it receives); it outputs to a.

By pushing the button once, the following pulses are sent:

```
button -low-> broadcaster
broadcaster -low-> a
broadcaster -low-> b
broadcaster -low-> c
a -high-> b
b -high-> c
c -high-> inv
inv -low-> a
a -low-> b
b -low-> c
c -low-> inv
inv -high-> a
```

After this sequence, the flip-flop modules all end up off, so pushing the button again repeats the same sequence.

Here's a more interesting example:

```
broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output
```

This module configuration includes the broadcaster, two flip-flops (named a and b), a single-input conjunction module (inv), a multi-input conjunction module (con), and an untyped module named output (for testing purposes). The multi-input conjunction module con watches the two flip-flop modules and, if they're both on, sends a low pulse to the output module.

Here's what happens if you push the button once:

```
button -low-> broadcaster
broadcaster -low-> a
a -high-> inv
a -high-> con
inv -low-> b
con -high-> output
b -high-> con
con -low-> output
```

Both flip-flops turn on and a low pulse is sent to output! However, now that both flip-flops are on and con remembers a high pulse from each of its two inputs, pushing the button a second time does something different:

```
button -low-> broadcaster
broadcaster -low-> a
a -low-> inv
a -low-> con
inv -high-> b
con -high-> output
```

Flip-flop a turns off! Now, con remembers a low pulse from module a, and so it sends only a high pulse to output.

Push the button a third time:

```
button -low-> broadcaster
broadcaster -low-> a
a -high-> inv
a -high-> con
inv -low-> b
con -low-> output
b -low-> con
con -high-> output
```

This time, flip-flop a turns on, then flip-flop b turns off. However, before b can turn off, the pulse sent to con is handled first, so it briefly remembers all high pulses for its inputs and sends a low pulse to output. After that, flip-flop b turns off, which causes con to update its state and send a high pulse to output.

Finally, with a on and b off, push the button a fourth time:

```
button -low-> broadcaster
broadcaster -low-> a
a -low-> inv
a -low-> con
inv -high-> b
con -high-> output
```

This completes the cycle: a turns off, causing con to remember only low pulses and restoring all modules to their original states.

To get the cables warmed up, the Elves have pushed the button 1000 times. How many pulses got sent as a result (including the pulses sent by the button itself)?

In the first example, the same thing happens every time the button is pushed: 8 low pulses and 4 high pulses are sent. So, after pushing the button 1000 times, 8000 low pulses and 4000 high pulses are sent. Multiplying these together gives 32000000.

In the second example, after pushing the button 1000 times, 4250 low pulses and 2750 high pulses are sent. Multiplying these together gives 11687500.

Consult your module configuration; determine the number of low pulses and high pulses that would be sent after pushing the button 1000 times, waiting for all pulses to be fully handled after each push of the button. What do you get if you multiply the total number of low pulses sent by the total number of high pulses sent?

button -low-> broadcaster


Step 0:
-------
broadcaster -low-> a
broadcaster -low-> b
broadcaster -low-> c

Step 1:
-------
a -high-> b
b -high-> c
c -high-> inv      history van inv = [hi]  -->  lo pulse

Step 2:
-------
inv -low-> a       

Step 3:
-------
a -low-> b

Step 4:
-------
b -low-> c

Step 5:
-------
c -low-> inv        history van inv = [hi, lo]  --> hi pulse

Step 6:
-------
inv -high-> a




Push #1
-------
button -low-> broadcaster
broadcaster -low-> a
a -high-> inv           history inv [hi]  --> lo pulse
a -high-> con           history con [hi]  --> lo pulse
inv -low-> b
con -high-> output
b -high-> con           history con [hi, hi]  --> lo pulse
con -low-> output

Push #2
-------
button -low-> broadcaster
broadcaster -low-> a
a -low-> inv            history inv [hi, lo]  --> hi pulse
a -low-> con            history con [hi, hi, lo]  --> hi pulse
inv -high-> b           
con -high-> output      

Push #3
-------
button -low-> broadcaster
broadcaster -low-> a
a -high-> inv          history inv [hi, lo, hi]  --> hi pulse
a -high-> con
inv -low-> b           lo pulse???
con -low-> output
b -low-> con
con -high-> output

### Solution

In [25]:
class FlipFlop:
    
    def __init__(self, name:str, output:str) -> None:
        self.name = name
        self.output = output
        self.on = False
        
    def process(self, pulse):
        if pulse == 'hi':
            return
        elif pulse == 'lo':
            if self.on:
                self.on = False
                return 'lo'
            else:
                self.on = True
                return 'hi'
            

class Conjunction:
    
    def __init__(self, name:str, output:str) -> None:
        self.name = name
        self.output = output
        self.history = []
        
    def process(self, pulse):
        self.history.append(pulse)
        if 'lo' in self.history:
            return 'hi'
        else:
            return 'lo'
    
    

In [26]:
def send_pulses(pulses, modules_map):
    pulses_out = []
    for _from, pulse, _to in pulses:  
        for name in _to.split(', '):
            module = modules_map[name]
            name_out = module.output
            pulse_out = module.process(pulse)
            if pulse_out and name_out != 'output':
                pulses_out.append((name, pulse_out, name_out))
    return pulses_out


def push_button(pulses, modules_map):
    print(pulses)
    while pulses:
        pulses = send_pulses(pulses, modules_map)
        print(pulses)

In [27]:
def my_func(input_string):
    modules_map = {}
    for line in input_string.splitlines():
        if 'broadcaster' in line:
            broadcast = line.split(' -> ')[1].split(', ')
        else:
            module_data = line.split(' -> ')
            module_name = module_data[0][1:]
            module_output = module_data[1]
            
            module_type = module_data[0][0]
            if module_type == '%':
                module = FlipFlop(module_name, module_output)
            elif module_type == '&':
                module = Conjunction(module_name, module_output)
                
            modules_map[module_name] = module
    
    pulses = [('B', 'lo', module_name) for module_name in broadcast]
    push_button(pulses, modules_map)
    print()    
    push_button(pulses, modules_map)
    print()    
    push_button(pulses, modules_map)

### Example

In [28]:
input_string = """broadcaster -> a, b, c
%a -> b
%b -> c
%c -> inv
&inv -> a"""

my_func(input_string)


[('B', 'lo', 'a'), ('B', 'lo', 'b'), ('B', 'lo', 'c')]
[('a', 'hi', 'b'), ('b', 'hi', 'c'), ('c', 'hi', 'inv')]
[('inv', 'lo', 'a')]
[('a', 'lo', 'b')]
[('b', 'lo', 'c')]
[('c', 'lo', 'inv')]
[('inv', 'hi', 'a')]
[]

[('B', 'lo', 'a'), ('B', 'lo', 'b'), ('B', 'lo', 'c')]
[('a', 'hi', 'b'), ('b', 'hi', 'c'), ('c', 'hi', 'inv')]
[('inv', 'hi', 'a')]
[]

[('B', 'lo', 'a'), ('B', 'lo', 'b'), ('B', 'lo', 'c')]
[('a', 'lo', 'b'), ('b', 'lo', 'c'), ('c', 'lo', 'inv')]
[('b', 'hi', 'c'), ('c', 'hi', 'inv'), ('inv', 'hi', 'a')]
[('inv', 'hi', 'a')]
[]


In [29]:
input_string = """broadcaster -> a
%a -> inv, con
&inv -> b
%b -> con
&con -> output"""

my_func(input_string)

[('B', 'lo', 'a')]
[('a', 'hi', 'inv, con')]
[('inv', 'lo', 'b')]
[('b', 'hi', 'con')]
[]

[('B', 'lo', 'a')]
[('a', 'lo', 'inv, con')]
[('inv', 'hi', 'b')]
[]

[('B', 'lo', 'a')]
[('a', 'hi', 'inv, con')]
[('inv', 'hi', 'b')]
[]


### Submission

In [ ]:
with open('input/day00.txt', 'r') as file:
    input_string = file.read()

my_func(input_string)


## Part 2

text

### Solution

In [ ]:
def my_func(input_string):
    pass


### Example

In [ ]:
input_string = """"""


my_func(input_string)


### Submission

In [ ]:
with open('input/day00.txt', 'r') as file:
    input_string = file.read()


my_func(input_string)
